<a href="https://colab.research.google.com/github/PurvaNatoo/Equity-Finance-Helper-LLM/blob/main/Equity_Analyst_Helper_LLM_Langchain_%2B_LLama_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Equity Analyst Helper LLM
*   Langchain
*   LLama embedding

### Text Loader

In [ ]:
!pip install faiss-cpu
!pip install sentence-transformers
!pip install accelerate
!pip install langchain
!pip install unstructured libmagic python-magic
!pip install torch

In [ ]:
exit()

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from langchain.document_loaders import UnstructuredURLLoader

In [ ]:
urls = []
for i in range(3):
    url = input("Enter URL " + str(i))
    urls.append(url)

Enter URL 0https://www.cnbc.com/2023/11/07/wonder-group-food-delivery-startup-gets-100-million-nestle-investment.html
Enter URL 1https://www.cnbc.com/2023/11/07/wonder-group-food-delivery-startup-gets-100-million-nestle-investment.html
Enter URL 2https://www.cnbc.com/2023/11/07/wonder-group-food-delivery-startup-gets-100-million-nestle-investment.html


In [ ]:
loader = UnstructuredURLLoader(urls=["https://www.cnbc.com/2023/11/07/wonder-group-food-delivery-startup-gets-100-million-nestle-investment.html"])
data = loader.load()

### Text Splitting

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=0
)

chunks = splitter.split_documents(data)

In [ ]:
len(chunks)

70

###Vector Database and Llama embeddings

In [ ]:
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from huggingface_hub import notebook_login
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain import HuggingFacePipeline
from langchain.text_splitter import CharacterTextSplitter
import textwrap
import sys

In [ ]:
#Replace YOUR_KEY with Huggingface LLama access key
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", token="YOUR_KEY")

In [ ]:
#Replace YOUR_KEY with Huggingface LLama access key
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             token="YOUR_KEY",
                                             device_map = "auto"
                                             )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [ ]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                max_new_tokens = 1024,
                do_sample=True,
                top_k=3,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [ ]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',model_kwargs={'device': 'cuda'})

In [ ]:
vectorstore=FAISS.from_documents(chunks, embeddings)

In [ ]:
chain =  RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore.as_retriever())


In [ ]:
chain

### Query

In [ ]:
query = "Can you summarize the text for me please"
result=chain({"query": query}, return_only_outputs=True)
wrapped_text = textwrap.fill(result['result'], width=500)
wrapped_text

### Notes:
* To retrieve useful data from LLM there are 2 methods:

  1. Stuff Method - Get all the chunks and combine them together and feed to the LLM. However the combined text chunk can exceed the LLM vector limit

  2. Map-reduce Method: Use individual LLM to process each chunk. More LLM calls are made